In [2]:
from gurobipy import *
from math import sqrt
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [7]:

# Reading the data file and preparting arrays
data1 =  pd.read_csv('transportation.csv')
print(data1)

data = data1.values
print(data)
header = data1.columns.values
print(header)

supply_nodes = data[1:, 1]
s = data[1:, 0]
print(supply_nodes)
print(s)

demand_nodes = data[0, 2:]
d = header[2:]
print(demand_nodes)
d = d.astype(np.float)
print(d)

c = data[1:, 2:]
print(c)

print(supply_nodes)
print(s)
print(multidict(zip(supply_nodes, s) ))
# Converting arrays to dictionaries
supply_nodes, s = multidict(zip(supply_nodes, s) )
demand_nodes, d = multidict(zip(demand_nodes, d) )

cost = {}
for i in range(len(supply_nodes)):
    for j in range(len(demand_nodes)):
        cost[supply_nodes[i], demand_nodes[j]] = c[i,j]

print(cost)

# Preapring an Optimization Model
tp = Model('transportation')

# Declaring variables
x = tp.addVars(cost, name = "transport")
print(x)

# Setting the objective
tp.setObjective( quicksum(cost[i,j] * x[i,j] for i in supply_nodes for j in demand_nodes), GRB.MINIMIZE)

# Adding constraints
tp.addConstrs(quicksum(x[i,j] for j in demand_nodes) == s[i] for i in supply_nodes )
tp.addConstrs(quicksum(x[i,j] for i in supply_nodes) == d[j] for j in demand_nodes )

# Solving the optimization problem
tp.optimize()

# Printing the optimal solutions obtained
p = tp.getAttr('x', x)
print("Optimal Solutions:")
for i in supply_nodes:
    for j in demand_nodes:
        print("\n %s to %s amount: %g" % (i, j , p[i,j]))
    

# Get objective value for future usage
print("\nOptimal value: \t%g" % tp.objVal)

   Unnamed: 0 Unnamed: 1       15      12    13
0         NaN        NaN  Chicago  Denver  Erie
1        15.0     Austin       10       7     9
2        25.0    Buffalo        4       9     8
[[nan nan 'Chicago' 'Denver' 'Erie']
 [15.0 'Austin' '10' '7' '9']
 [25.0 'Buffalo' '4' '9' '8']]
['Unnamed: 0' 'Unnamed: 1' '15' '12' '13']
['Austin' 'Buffalo']
[15.0 25.0]
['Chicago' 'Denver' 'Erie']
[15. 12. 13.]
[['10' '7' '9']
 ['4' '9' '8']]
['Austin' 'Buffalo']
[15.0 25.0]
[['Austin', 'Buffalo'], {'Austin': 15.0, 'Buffalo': 25.0}]
{('Austin', 'Chicago'): '10', ('Austin', 'Denver'): '7', ('Austin', 'Erie'): '9', ('Buffalo', 'Chicago'): '4', ('Buffalo', 'Denver'): '9', ('Buffalo', 'Erie'): '8'}
{('Austin', 'Chicago'): <gurobi.Var *Awaiting Model Update*>, ('Austin', 'Denver'): <gurobi.Var *Awaiting Model Update*>, ('Austin', 'Erie'): <gurobi.Var *Awaiting Model Update*>, ('Buffalo', 'Chicago'): <gurobi.Var *Awaiting Model Update*>, ('Buffalo', 'Denver'): <gurobi.Var *Awaiting Model Update*>, 

In [10]:
# Data Preparation
data1 =  pd.read_csv('mincost.csv')
print(data1)

data = data1.values
print(type(data))
print(data)
header = data1.columns.values
print(header)

start_node = data[:,0]
end_node = data[:,1]
c = data[:,2]
u = data[:,3]


data2 = pd.read_csv('mincost_b.csv')
data2 = data2.values
print(data2)
b = data2[:,1]
# nodes, b = multidict(zip(nodes, b))
print(b)


# number of nodes and number of links
no_node = max(max(start_node), max(end_node) )
no_link = len(start_node)
print(no_node)
print(no_link)

# Creating a graph
nodes = np.arange(1, no_node+1)
print(nodes)
print(start_node)
links =  zip( start_node, end_node) 
links, c_dict, u_dict = multidict({(start_node[i], end_node[i]): [c[i], u[i]] for i in range(no_link)})
print(links)
print(c_dict)
print(u_dict)

# Preparing an optimization model
mcnf = Model('mincost')

# Defining decision variables
x = mcnf.addVars(links, ub = u_dict)



# Setting the objective
mcnf.setObjective(quicksum(c_dict[link] * x[link] for link in links), GRB.MINIMIZE)

# Adding the flow conservation constraints
mcnf.addConstrs(quicksum(x[link] for link in links.select(i,'*')) - quicksum(x[link] for link in links.select('*',i)) 
                == b[i] for i in nodes)

mcnf.optimize()

# Printing the optimal solutions obtained
p = mcnf.getAttr('x', x)
print("Optimal Solutions:")
for link in links:
    print("\n flow on link %s: %g" % (link , p[link]))

# Get objective value for future usage
print("\nOptimal value: \t%g" % mcnf.objVal)

   start node i  end node j  c_ij  u_ij
0             1           2     2   inf
1             1           3     5   inf
2             2           3     3   inf
3             3           4     1   inf
4             3           5     2   1.0
5             4           1     0   inf
6             4           5     2   inf
7             5           2     4   inf
<class 'numpy.ndarray'>
[[ 1.  2.  2. inf]
 [ 1.  3.  5. inf]
 [ 2.  3.  3. inf]
 [ 3.  4.  1. inf]
 [ 3.  5.  2.  1.]
 [ 4.  1.  0. inf]
 [ 4.  5.  2. inf]
 [ 5.  2.  4. inf]]
['start node i' 'end node j' 'c_ij' 'u_ij']
[[ 1 -5]
 [ 2 10]
 [ 3  0]
 [ 4 -2]
 [ 5 -3]]
[-5 10  0 -2 -3]
5.0
8
[1. 2. 3. 4. 5.]
[1. 1. 2. 3. 3. 4. 4. 5.]
<gurobi.tuplelist (8 tuples, 2 values each):
 ( 1.0 , 2.0 )
 ( 1.0 , 3.0 )
 ( 2.0 , 3.0 )
 ( 3.0 , 4.0 )
 ( 3.0 , 5.0 )
 ( 4.0 , 1.0 )
 ( 4.0 , 5.0 )
 ( 5.0 , 2.0 )
>
{(1.0, 2.0): 2.0, (1.0, 3.0): 5.0, (2.0, 3.0): 3.0, (3.0, 4.0): 1.0, (3.0, 5.0): 2.0, (4.0, 1.0): 0.0, (4.0, 5.0): 2.0, (5.0, 2.0): 4.0}
{(1

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices